## 使用Gensim判断两篇中文文章相似度
参考[gensim官网](https://radimrehurek.com/gensim)
>注：分词使用jieba分词

整体流程为：
1、预处理，分词与去停止词与标点
2、使用gensim构建语料库与模型
3、比较相似度

In [1]:
from gensim import corpora,models,similarities
import jieba
from pprint import pprint

C:\Anaconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 分词与去除停止词与标点

In [2]:
#读取数据，这里选取了豆瓣电影上的影评
delete_words="的 了 么 呢 吗 地 与 但 时 就 啊 《 》 “ ” … ！ ， 。".split()
texts=[[word for word in jieba.cut(document) if word not in delete_words] for document in open('gensim.txt')]
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
#去除只出现一次的词语，可选
texts = [[token for token in text if frequency[token] > 1]for text in texts]
pprint(texts)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\DANIEL~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.791 seconds.
Prefix dict has been built succesfully.


[['非常',
  '很',
  '有',
  '并',
  '改编',
  '说',
  '是',
  '电影',
  '戏',
  '最后',
  '晚餐',
  '构图',
  '不是',
  '是',
  '\n'],
 ['却', '改编', '用', '正义', '都', '很', '\n'],
 ['电影',
  '感',
  '很',
  '用',
  '出',
  '用',
  '用',
  '最后',
  '晚餐',
  '构图',
  '将',
  '正义',
  '也',
  '让',
  '道德',
  '这',
  '也',
  '是',
  '改编',
  '只有',
  '最后',
  '却',
  '将',
  '道德',
  '\n'],
 ['说',
  '是',
  '非常',
  '卡司',
  '很',
  '故事',
  '大菠萝',
  '戏',
  '和',
  '有',
  '大菠萝',
  '不',
  '效仿',
  '\n'],
 ['卡司', '和', '电影', '感', '也', '并', '不', '故事', '\n'],
 ['电影', '都', '效仿', '拍摄', '拍摄', '出', '是', '让', '不是', '感', '这', '只有']]


### 构建预料库与模型

In [3]:
#词典
dictionary = corpora.Dictionary(texts)
#语料库，将文档转换为bag of word的表示
corpus = [dictionary.doc2bow(text) for text in texts]
#构建LDA模型，选择3个潜藏主题
model  = models.LdaModel(corpus,id2word=dictionary,num_topics=3)
#显示潜藏主题
model.print_topics(3)

#将原语料库转换为lda模型的模型表示
vec_lda = model[corpus]
#每个文本都可以表示为一个3维向量表示，之后可以使用余弦距离计算相似度
print(vec_lda[0])

[(0, 0.022886620823319163), (1, 0.022548566279495936), (2, 0.95456481289718487)]


### 相似度比较与相似查询

In [11]:
#将原预料转换为lda空间并且建立索引
index = similarities.MatrixSimilarity(model[corpus])

#使用索引进行相似查询，结果是(document_number, document_similarity)的二元组
sims = index[vec_lda[0]]

#打印全部的结果
print(list(enumerate(sims)))

[(0, 1.0), (1, 0.075662106), (2, 0.039198291), (3, 0.99999934), (4, 0.99977362), (5, 0.053346336)]
